# 3.0 Feature Engineering

In [35]:
import numpy as np
import pandas as pd

## 3.1 Feature Creation

In [36]:
class PoseDimensionCalculator:
    
    def __init__(self, data, include_z=False):
        self.data = data
        
        self.enhance_pose_landmarks = self.enhance_pose_landmarks()
        self.data = self.enhance_pose_landmarks

        if include_z:
            distances = self.calculate_distances_z()
            self.data = pd.concat([self.data, distances], axis=1)
            angles = self.calculate_connected_joint_range_z()
            self.data = pd.concat([self.data, angles], axis=1)
        
        else:
            distances = self.calculate_distances()
            self.data = pd.concat([self.data, distances], axis=1)
            angles = self.calculate_connected_joint_range()
            self.data = pd.concat([self.data, angles], axis=1)

    def enhance_pose_landmarks(self):
        enhanced_data = []
        for index, row in self.data.iterrows():
            enhanced_row = self.process_row(row)
            enhanced_row['index'] = index
            enhanced_data.append(enhanced_row)
        enhanced_df = pd.DataFrame(enhanced_data).set_index('index')
        merged = pd.concat([self.data, enhanced_df], axis=1)
        return merged

    def process_row(self, row):
        x_cols = sorted([col for col in self.data.columns if col.endswith('_x')])
        y_cols = sorted([col for col in self.data.columns if col.endswith('_y')])
        z_cols = sorted([col for col in self.data.columns if col.endswith('_z')])        
        x = row[x_cols].values
        y = row[y_cols].values
        z = row[z_cols].values

        enhanced_row = {
            'head_x': x[0:11].mean(),
            'head_y': y[0:11].mean(), 
            'head_z': z[0:11].mean(),
            'chest_x': x[11:13].mean(),
            'chest_y': y[11:13].mean(), 
            'chest_z': z[11:13].mean(),
            'stomach_x': (x[11:13].mean() + x[23:25].mean()) / 2,
            'stomach_y': (y[11:13].mean() + y[23:25].mean()) / 2,
            'stomach_z': (z[11:13].mean() + z[23:25].mean()) / 2,
            'hip_x': x[23:25].mean(),
            'hip_y': y[23:25].mean(),
            'hip_z': z[23:25].mean(),
        }

        return enhanced_row

    def calculate_pose_distance(self, Ax, Ay, Bx, By):
        distance = np.sqrt((self.data[Ax] - self.data[Bx])**2 + (self.data[Ay] - self.data[By])**2)
        return np.round(distance, 4)

    def calculate_pose_distance_z(self, x1, y1, z1, x2, y2, z2):
        distance = np.sqrt((self.data[x2] - self.data[x1]) ** 2 +
                       (self.data[y2] - self.data[y1]) ** 2 +
                       (self.data[z2] - self.data[z1]) ** 2)
        return np.round(distance, 4)
        
    def calculate_distances(self):
        dist = pd.DataFrame(index=self.data.index)
        dist['d_nose_to_rgt_knee'] = self.calculate_pose_distance('landmark_00_x', 'landmark_00_y', 'landmark_25_x', 'landmark_25_y')
        dist['d_nose_to_lft_knee'] = self.calculate_pose_distance('landmark_00_x', 'landmark_00_y', 'landmark_26_x', 'landmark_26_y')
        dist['d_rgt_shoulder_to_ankle'] = self.calculate_pose_distance('landmark_11_x', 'landmark_11_y', 'landmark_25_x', 'landmark_25_y')
        dist['d_lft_shoulder_to_ankle'] = self.calculate_pose_distance('landmark_12_x', 'landmark_12_y', 'landmark_26_x', 'landmark_26_y')
        dist['d_rgt_shoulder_to_lft_ankle'] = self.calculate_pose_distance('landmark_11_x', 'landmark_11_y','landmark_26_x', 'landmark_26_y')
        dist['d_lft_shoulder_to_rgt_ankle'] = self.calculate_pose_distance('landmark_12_x', 'landmark_12_y','landmark_25_x', 'landmark_25_y')    
        dist['d_wrists'] = self.calculate_pose_distance('landmark_15_x', 'landmark_15_y', 'landmark_16_x', 'landmark_16_y')
        dist['d_elbows'] = self.calculate_pose_distance('landmark_13_x', 'landmark_13_y', 'landmark_14_x', 'landmark_14_y')
        dist['d_knees'] = self.calculate_pose_distance('landmark_25_x', 'landmark_25_y', 'landmark_26_x', 'landmark_26_y')
        dist['d_ankles'] = self.calculate_pose_distance('landmark_27_x', 'landmark_27_y', 'landmark_28_x', 'landmark_28_y')
        return dist

    def calculate_distances_z(self):
        dist = pd.DataFrame(index=self.data.index)
        dist['d_nose_to_rgt_knee'] = self.calculate_pose_distance_z('landmark_00_x', 'landmark_00_y','landmark_00_z','landmark_25_x', 'landmark_25_y','landmark_25_z')
        dist['d_nose_to_lft_knee'] = self.calculate_pose_distance_z('landmark_00_x', 'landmark_00_y','landmark_00_z','landmark_26_x', 'landmark_26_y','landmark_26_z')
        dist['d_rgt_shoulder_to_ankle'] = self.calculate_pose_distance_z('landmark_11_x', 'landmark_11_y','landmark_11_z', 'landmark_25_x', 'landmark_25_y','landmark_25_z')
        dist['d_lft_shoulder_to_ankle'] = self.calculate_pose_distance_z('landmark_12_x', 'landmark_12_y','landmark_12_z','landmark_26_x', 'landmark_26_y','landmark_26_z')
        dist['d_rgt_shoulder_to_lft_ankle'] = self.calculate_pose_distance_z('landmark_11_x', 'landmark_11_y','landmark_11_z', 'landmark_26_x', 'landmark_26_y','landmark_26_z')
        dist['d_lft_shoulder_to_rgt_ankle'] = self.calculate_pose_distance_z('landmark_12_x', 'landmark_12_y','landmark_12_z','landmark_25_x', 'landmark_25_y','landmark_25_z')
        dist['d_wrists'] = self.calculate_pose_distance_z('landmark_15_x', 'landmark_15_y','landmark_15_z', 'landmark_16_x', 'landmark_16_y','landmark_16_z')
        dist['d_elbows'] = self.calculate_pose_distance_z('landmark_13_x', 'landmark_13_y','landmark_13_z','landmark_14_x', 'landmark_14_y','landmark_14_z')
        dist['d_knees'] = self.calculate_pose_distance_z('landmark_25_x', 'landmark_25_y','landmark_25_z', 'landmark_26_x', 'landmark_26_y','landmark_26_z')
        dist['d_ankles'] = self.calculate_pose_distance_z('landmark_27_x', 'landmark_27_y','landmark_27_z','landmark_28_x', 'landmark_28_y','landmark_28_z')
        return dist
        
    def calculate_pose_angle(self, Ax, Ay, Bx, By, Cx, Cy):
        A = self.data[[Ax, Ay]].values
        B = self.data[[Bx, By]].values
        C = self.data[[Cx, Cy]].values
        BA = A - B
        BC = C - B
        angle_BA = np.arctan2(BA[:, 1], BA[:, 0])
        angle_BC = np.arctan2(BC[:, 1], BC[:, 0])
        angle_difference = np.degrees(angle_BC - angle_BA)    
        return np.round(angle_difference, 4)

    def calculate_pose_angle_z(self, Ax, Ay, Az, Bx, By, Bz, Cx, Cy, Cz):
        A = self.data[[Ax, Ay, Az]].values
        B = self.data[[Bx, By, Bz]].values
        C = self.data[[Cx, Cy, Cz]].values
        BA = A - B
        BC = C - B
        cosine_angle = (BA * BC).sum(axis=1) / (np.linalg.norm(BA, axis=1) * np.linalg.norm(BC, axis=1))
        angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))  # Clip to handle floating point errors
        angle_difference = np.degrees(angle)  # Convert to degrees
        return np.round(angle_difference, 4)    
        
    def calculate_connected_joint_range(self):
        angles = pd.DataFrame(index=self.data.index)
        angles['a_nose_to_rgt_shoulder'] = self.calculate_pose_angle('landmark_00_x', 'landmark_00_y', 'chest_x', 'chest_y','landmark_11_x','landmark_11_y')
        angles['a_nose_to_lft_shoulder'] = self.calculate_pose_angle('landmark_00_x', 'landmark_00_y', 'chest_x', 'chest_y','landmark_12_x','landmark_12_y')
        angles['a_mid_hip_to_knees'] = self.calculate_pose_angle('landmark_26_x','landmark_26_y','hip_x', 'hip_y','landmark_25_x','landmark_25_y')
        angles['a_lft_shoulder_to_wrist'] = self.calculate_pose_angle('landmark_12_x', 'landmark_12_y', 'landmark_14_x', 'landmark_14_y', 'landmark_16_x', 'landmark_16_y')
        angles['a_lft_hip_to_ankle'] = self.calculate_pose_angle('landmark_24_x', 'landmark_24_y', 'landmark_26_x', 'landmark_26_y','landmark_28_x', 'landmark_28_y')
        angles['a_rgt_shoulder_to_wrist'] = self.calculate_pose_angle('landmark_11_x', 'landmark_11_y', 'landmark_13_x', 'landmark_13_y', 'landmark_15_x', 'landmark_15_y')
        angles['a_rgt_hip_to_ankle'] = self.calculate_pose_angle('landmark_23_x', 'landmark_23_y', 'landmark_25_x', 'landmark_25_y','landmark_27_x', 'landmark_27_y')
        return angles

    def calculate_connected_joint_range_z(self):
        angles = pd.DataFrame(index=self.data.index)
        angles['a_nose_to_rgt_shoulder'] = self.calculate_pose_angle_z('landmark_00_x', 'landmark_00_y','landmark_00_z','chest_x','chest_y','chest_z','landmark_11_x','landmark_11_y','landmark_11_z')
        angles['a_nose_to_lft_shoulder'] = self.calculate_pose_angle_z('landmark_00_x', 'landmark_00_y','landmark_00_z','chest_x','chest_y','chest_z','landmark_12_x','landmark_12_y','landmark_12_z')
        angles['a_mid_hip_to_knees'] = self.calculate_pose_angle_z('landmark_26_x', 'landmark_26_y','landmark_26_z','hip_x','hip_y','hip_z','landmark_25_x','landmark_25_y','landmark_25_z')
        angles['a_lft_shoulder_to_wrist'] = self.calculate_pose_angle_z('landmark_12_x', 'landmark_12_y','landmark_12_z','landmark_14_x','landmark_14_y','landmark_14_z','landmark_16_x','landmark_16_y','landmark_16_z')        
        angles['a_lft_hip_to_ankle'] = self.calculate_pose_angle_z('landmark_24_x', 'landmark_24_y','landmark_24_z','landmark_26_x','landmark_26_y','landmark_26_z','landmark_28_x','landmark_28_y','landmark_28_z')
        angles['a_rgt_shoulder_to_wrist'] = self.calculate_pose_angle_z('landmark_11_x', 'landmark_11_y','landmark_11_z','landmark_13_x','landmark_13_y','landmark_13_z','landmark_15_x','landmark_15_y','landmark_15_z')        
        angles['a_rgt_hip_to_ankle'] = self.calculate_pose_angle_z('landmark_23_x', 'landmark_23_y','landmark_23_z','landmark_25_x','landmark_25_y','landmark_25_z','landmark_27_x','landmark_27_y','landmark_27_z')
        return angles

### Feature Creations for Training Data

In [30]:
raw_data = pd.read_csv("data/training/pose_data_raw.csv")
results = PoseDimensionCalculator(raw_data,include_z=False)
results.data

,image_filename,pose_name,landmark_00_x,landmark_00_y,landmark_00_z,landmark_00_v,landmark_01_x,landmark_01_y,landmark_01_z,landmark_01_v,...,d_elbows,d_knees,d_ankles,a_nose_to_rgt_shoulder,a_nose_to_lft_shoulder,a_mid_hip_to_knees,a_lft_shoulder_to_wrist,a_lft_hip_to_ankle,a_rgt_shoulder_to_wrist,a_rgt_hip_to_ankle
0,armpit-ankle-hold-dart-lft.png,armpit-ankle-hold-dart-lft,0.156662,0.145272,-0.226705,0.999967,0.176508,0.134873,-0.255778,0.999960,...,0.0596,0.3454,0.5420,114.5971,294.5971,-107.1103,185.5112,148.0273,-174.2554,176.5460
1,armpit-ankle-hold-dart-rgt.png,armpit-ankle-hold-dart-rgt,0.841027,0.155245,-0.178235,0.999999,0.825823,0.140992,-0.144594,0.999998,...,0.0841,0.3446,0.5437,57.3588,-122.6412,-107.2030,-180.4158,181.7798,177.1881,211.8761
2,armpit-ankle-hold-inverted-side-split-lft.png,armpit-ankle-hold-inverted-side-split-lft,0.579417,0.602999,-0.569200,0.999869,0.582305,0.620670,-0.547844,0.999827,...,0.3325,0.3516,0.6082,-271.1543,-91.1543,162.1096,175.7817,-173.4181,-81.7210,180.5491
3,armpit-ankle-hold-inverted-side-split-rgt.png,armpit-ankle-hold-inverted-side-split-rgt,0.416769,0.602737,-0.523792,0.999341,0.396236,0.613743,-0.502519,0.998980,...,0.3427,0.3424,0.6087,-275.0607,-95.0607,-198.5347,-305.4387,181.6454,174.4563,-182.7760
4,armpit-elbow-hold-inverted-straddle-lft.png,armpit-elbow-hold-inverted-straddle-lft,0.505175,0.451810,-0.427546,0.999929,0.497041,0.466059,-0.436100,0.999883,...,0.1287,0.3876,0.7241,268.9779,88.9779,161.5524,49.2880,179.9178,-136.4642,176.4471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,wrist-sit-rgt.png,wrist-sit-rgt,0.506111,0.534306,-0.154565,0.999970,0.509519,0.520750,-0.135237,0.999945,...,0.2341,0.4005,0.7233,65.8490,245.8490,122.2358,116.8475,-185.4802,-185.5553,-178.7115
540,yogini-lft.png,yogini-lft,0.750999,0.265975,0.050217,1.000000,0.734617,0.252835,0.082880,1.000000,...,0.0377,0.0409,0.0833,210.6862,30.6862,-7.5683,186.1655,-70.8528,187.0363,-81.5335
541,yogini-rgt.png,yogini-rgt,0.254569,0.253966,0.185102,0.999985,0.276668,0.244936,0.168699,0.999988,...,0.0300,0.0258,0.0578,-16.2186,163.7814,-4.8821,168.7918,80.8230,173.0351,74.3108
542,yogini-twisted-lft.png,yogini-twisted-lft,0.356109,0.269642,-0.166786,0.999335,0.358961,0.250586,-0.201648,0.998349,...,0.1726,0.3890,0.5624,104.0931,284.0931,-138.9177,229.7118,121.3195,-182.1522,66.5343


In [31]:
results.data.to_csv("data/training/pose_data_raw_res.csv", index=False)

In [27]:
aug_data = pd.read_csv("data/training/pose_data_aug.csv")
results_2d = PoseDimensionCalculator(aug_data,include_z=False)
results_2d.data

,image_filename,label,label_encoded,theta,landmark_00_x,landmark_00_y,landmark_00_z,landmark_01_x,landmark_01_y,landmark_01_z,...,d_elbows,d_knees,d_ankles,a_nose_to_rgt_shoulder,a_nose_to_lft_shoulder,a_mid_hip_to_knees,a_lft_shoulder_to_wrist,a_lft_hip_to_ankle,a_rgt_shoulder_to_wrist,a_rgt_hip_to_ankle
0,armpit-ankle-hold-dart-lft.png,armpit-ankle-hold-dart,0,0,0.156662,0.854728,-0.226705,0.176508,0.865127,-0.255778,...,0.0596,0.3454,0.5420,-114.5971,-294.5971,107.1103,-185.5112,-148.0273,174.2554,-176.5460
1,armpit-ankle-hold-dart-lft.png,armpit-ankle-hold-dart,0,10,0.193649,0.854728,-0.196057,0.218242,0.865127,-0.221242,...,0.1809,0.3887,0.5748,-129.4264,-309.4264,113.6522,-183.2347,-158.9269,175.4845,-179.2226
2,armpit-ankle-hold-dart-lft.png,armpit-ankle-hold-dart,0,20,0.224752,0.854728,-0.159452,0.253345,0.865127,-0.179984,...,0.3086,0.4219,0.5941,-127.1438,-307.1438,118.3787,-181.7597,-170.5934,176.1871,-184.0612
3,armpit-ankle-hold-dart-lft.png,armpit-ankle-hold-dart,0,30,0.249026,0.854728,-0.118002,0.280750,0.865127,-0.133256,...,0.4284,0.4434,0.5990,-121.2188,-301.2188,121.5642,-180.5878,-182.3835,176.6125,-195.1985
4,armpit-ankle-hold-dart-lft.png,armpit-ankle-hold-dart,0,40,0.265733,0.854728,-0.072966,0.299624,0.865127,-0.082480,...,0.5356,0.4526,0.5893,-112.4241,-292.4241,123.4006,-179.4389,-193.7460,176.8675,-226.3108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19579,yogini-twisted-rgt.png,yogini-twisted,271,310,0.221597,0.731075,-0.655868,0.241815,0.751349,-0.633527,...,0.5911,0.5505,0.6199,-58.2836,121.7164,154.3500,172.0633,31.9434,-228.7234,-201.5062
19580,yogini-twisted-rgt.png,yogini-twisted,271,320,0.332121,0.731075,-0.607424,0.348152,0.751349,-0.581912,...,0.5291,0.5494,0.6267,-39.9066,140.0934,154.1885,174.1254,36.1317,-214.9999,-209.2061
19581,yogini-twisted-rgt.png,yogini-twisted,271,330,0.432553,0.731075,-0.540524,0.443911,0.751349,-0.512615,...,0.4541,0.5319,0.6252,-32.2659,147.7341,153.2878,175.4786,41.6074,-199.8780,-216.7328
19582,yogini-twisted-rgt.png,yogini-twisted,271,340,0.519843,0.731075,-0.457200,0.526182,0.751349,-0.427743,...,0.3702,0.4986,0.6154,-30.2609,149.7391,151.5254,176.4539,48.8414,-183.9328,-224.2409


In [28]:
results_2d.data.to_csv("data/training/pose_data_aug_res1.csv", index=False)

**3D Approach**

In [39]:
aug_data = pd.read_csv("data/training/pose_data_aug.csv")
results_3d = PoseDimensionCalculator(aug_data,include_z=True)
results_3d.data

,image_filename,label,label_encoded,theta,landmark_00_x,landmark_00_y,landmark_00_z,landmark_01_x,landmark_01_y,landmark_01_z,...,d_elbows,d_knees,d_ankles,a_nose_to_rgt_shoulder,a_nose_to_lft_shoulder,a_mid_hip_to_knees,a_lft_shoulder_to_wrist,a_lft_hip_to_ankle,a_rgt_shoulder_to_wrist,a_rgt_hip_to_ankle
0,armpit-ankle-hold-dart-lft.png,armpit-ankle-hold-dart,0,0,0.156662,0.854728,-0.226705,0.176508,0.865127,-0.255778,...,0.7847,0.4529,0.5992,67.5579,112.4421,121.3078,118.9944,119.0912,177.0366,141.0389
1,armpit-ankle-hold-dart-lft.png,armpit-ankle-hold-dart,0,10,0.193649,0.854728,-0.196057,0.218242,0.865127,-0.221242,...,0.7847,0.4529,0.5992,67.5579,112.4421,121.3078,118.9944,119.0912,177.0366,141.0389
2,armpit-ankle-hold-dart-lft.png,armpit-ankle-hold-dart,0,20,0.224752,0.854728,-0.159452,0.253345,0.865127,-0.179984,...,0.7847,0.4529,0.5992,67.5579,112.4421,121.3078,118.9944,119.0912,177.0366,141.0389
3,armpit-ankle-hold-dart-lft.png,armpit-ankle-hold-dart,0,30,0.249026,0.854728,-0.118002,0.280750,0.865127,-0.133256,...,0.7847,0.4529,0.5992,67.5579,112.4421,121.3078,118.9944,119.0912,177.0366,141.0389
4,armpit-ankle-hold-dart-lft.png,armpit-ankle-hold-dart,0,40,0.265733,0.854728,-0.072966,0.299624,0.865127,-0.082480,...,0.7847,0.4529,0.5992,67.5579,112.4421,121.3078,118.9944,119.0912,177.0366,141.0389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19579,yogini-twisted-rgt.png,yogini-twisted,271,310,0.221597,0.731075,-0.655868,0.241815,0.751349,-0.633527,...,0.6759,0.5520,0.6271,116.5380,63.4620,154.1874,159.0989,59.3940,98.2172,120.2231
19580,yogini-twisted-rgt.png,yogini-twisted,271,320,0.332121,0.731075,-0.607424,0.348152,0.751349,-0.581912,...,0.6759,0.5520,0.6271,116.5380,63.4620,154.1874,159.0989,59.3940,98.2172,120.2231
19581,yogini-twisted-rgt.png,yogini-twisted,271,330,0.432553,0.731075,-0.540524,0.443911,0.751349,-0.512615,...,0.6759,0.5520,0.6271,116.5380,63.4620,154.1874,159.0989,59.3940,98.2172,120.2231
19582,yogini-twisted-rgt.png,yogini-twisted,271,340,0.519843,0.731075,-0.457200,0.526182,0.751349,-0.427743,...,0.6759,0.5520,0.6271,116.5380,63.4620,154.1874,159.0989,59.3940,98.2172,120.2231


In [40]:
results_3d.data.to_csv("data/training/pose_data_aug_res2.csv", index=False)

### Feature Creations for Test Dataset

**2d Approach**

In [37]:
processed_dir = "data/test/processed/superman/01/"
processed_data = pd.read_csv(f"{processed_dir}/pose_data_raw.csv")
y_cols = [col for col in processed_data.columns if col.endswith('_y')]
processed_data[y_cols] = 1 - processed_data[y_cols]
new_data = PoseDimensionCalculator(processed_data,include_z=False)
new_data.data.to_csv(f"{processed_dir}/pose_data_res1.csv", index=False)

processed_dir = "data/test/processed/superman/02/"
processed_data = pd.read_csv(f"{processed_dir}/pose_data_raw.csv")
y_cols = [col for col in processed_data.columns if col.endswith('_y')]
processed_data[y_cols] = 1 - processed_data[y_cols]
new_data = PoseDimensionCalculator(processed_data,include_z=False)
new_data.data.to_csv(f"{processed_dir}/pose_data_res1.csv", index=False)

processed_dir = "data/test/processed/superman/03/"
processed_data = pd.read_csv(f"{processed_dir}/pose_data_raw.csv")
y_cols = [col for col in processed_data.columns if col.endswith('_y')]
processed_data[y_cols] = 1 - processed_data[y_cols]
new_data = PoseDimensionCalculator(processed_data,include_z=False)
new_data.data.to_csv(f"{processed_dir}/pose_data_res1.csv", index=False)

processed_dir = "data/test/processed/crucifix/01/"
processed_data = pd.read_csv(f"{processed_dir}/pose_data_raw.csv")
y_cols = [col for col in processed_data.columns if col.endswith('_y')]
processed_data[y_cols] = 1 - processed_data[y_cols]
new_data = PoseDimensionCalculator(processed_data,include_z=False)
new_data.data.to_csv(f"{processed_dir}/pose_data_res1.csv", index=False)

processed_dir = "data/test/processed/crucifix/02/"
processed_data = pd.read_csv(f"{processed_dir}/pose_data_raw.csv")
y_cols = [col for col in processed_data.columns if col.endswith('_y')]
processed_data[y_cols] = 1 - processed_data[y_cols]
new_data = PoseDimensionCalculator(processed_data,include_z=False)
new_data.data.to_csv(f"{processed_dir}/pose_data_res1.csv", index=False)

processed_dir = "data/test/processed/crucifix/03/"
processed_data = pd.read_csv(f"{processed_dir}/pose_data_raw.csv")
y_cols = [col for col in processed_data.columns if col.endswith('_y')]
processed_data[y_cols] = 1 - processed_data[y_cols]
new_data = PoseDimensionCalculator(processed_data,include_z=False)
new_data.data.to_csv(f"{processed_dir}/pose_data_res1.csv", index=False)

processed_dir = "data/test/processed/fireman/01/"
processed_data = pd.read_csv(f"{processed_dir}/pose_data_raw.csv")
y_cols = [col for col in processed_data.columns if col.endswith('_y')]
processed_data[y_cols] = 1 - processed_data[y_cols]
new_data = PoseDimensionCalculator(processed_data,include_z=False)
new_data.data.to_csv(f"{processed_dir}/pose_data_res1.csv", index=False)

processed_dir = "data/test/processed/fireman/02/"
processed_data = pd.read_csv(f"{processed_dir}/pose_data_raw.csv")
y_cols = [col for col in processed_data.columns if col.endswith('_y')]
processed_data[y_cols] = 1 - processed_data[y_cols]
new_data = PoseDimensionCalculator(processed_data,include_z=False)
new_data.data.to_csv(f"{processed_dir}/pose_data_res1.csv", index=False)

processed_dir = "data/test/processed/fireman/03/"
processed_data = pd.read_csv(f"{processed_dir}/pose_data_raw.csv")
y_cols = [col for col in processed_data.columns if col.endswith('_y')]
processed_data[y_cols] = 1 - processed_data[y_cols]
new_data = PoseDimensionCalculator(processed_data,include_z=False)
new_data.data.to_csv(f"{processed_dir}/pose_data_res1.csv", index=False)

**3d Approach**

In [38]:
processed_dir = "data/test/processed/superman/01/"
processed_data = pd.read_csv(f"{processed_dir}/pose_data_raw.csv")
y_cols = [col for col in processed_data.columns if col.endswith('_y')]
processed_data[y_cols] = 1 - processed_data[y_cols]
new_data = PoseDimensionCalculator(processed_data,include_z=True)
new_data.data.to_csv(f"{processed_dir}/pose_data_res2.csv", index=False)

processed_dir = "data/test/processed/superman/02/"
processed_data = pd.read_csv(f"{processed_dir}/pose_data_raw.csv")
y_cols = [col for col in processed_data.columns if col.endswith('_y')]
processed_data[y_cols] = 1 - processed_data[y_cols]
new_data = PoseDimensionCalculator(processed_data,include_z=True)
new_data.data.to_csv(f"{processed_dir}/pose_data_res2.csv", index=False)

processed_dir = "data/test/processed/superman/03/"
processed_data = pd.read_csv(f"{processed_dir}/pose_data_raw.csv")
y_cols = [col for col in processed_data.columns if col.endswith('_y')]
processed_data[y_cols] = 1 - processed_data[y_cols]
new_data = PoseDimensionCalculator(processed_data,include_z=True)
new_data.data.to_csv(f"{processed_dir}/pose_data_res2.csv", index=False)

processed_dir = "data/test/processed/crucifix/01/"
processed_data = pd.read_csv(f"{processed_dir}/pose_data_raw.csv")
y_cols = [col for col in processed_data.columns if col.endswith('_y')]
processed_data[y_cols] = 1 - processed_data[y_cols]
new_data = PoseDimensionCalculator(processed_data,include_z=True)
new_data.data.to_csv(f"{processed_dir}/pose_data_res2.csv", index=False)

processed_dir = "data/test/processed/crucifix/02/"
processed_data = pd.read_csv(f"{processed_dir}/pose_data_raw.csv")
y_cols = [col for col in processed_data.columns if col.endswith('_y')]
processed_data[y_cols] = 1 - processed_data[y_cols]
new_data = PoseDimensionCalculator(processed_data,include_z=True)
new_data.data.to_csv(f"{processed_dir}/pose_data_res2.csv", index=False)

processed_dir = "data/test/processed/crucifix/03/"
processed_data = pd.read_csv(f"{processed_dir}/pose_data_raw.csv")
y_cols = [col for col in processed_data.columns if col.endswith('_y')]
processed_data[y_cols] = 1 - processed_data[y_cols]
new_data = PoseDimensionCalculator(processed_data,include_z=True)
new_data.data.to_csv(f"{processed_dir}/pose_data_res2.csv", index=False)

processed_dir = "data/test/processed/fireman/01/"
processed_data = pd.read_csv(f"{processed_dir}/pose_data_raw.csv")
y_cols = [col for col in processed_data.columns if col.endswith('_y')]
processed_data[y_cols] = 1 - processed_data[y_cols]
new_data = PoseDimensionCalculator(processed_data,include_z=True)
new_data.data.to_csv(f"{processed_dir}/pose_data_res2.csv", index=False)

processed_dir = "data/test/processed/fireman/02/"
processed_data = pd.read_csv(f"{processed_dir}/pose_data_raw.csv")
y_cols = [col for col in processed_data.columns if col.endswith('_y')]
processed_data[y_cols] = 1 - processed_data[y_cols]
new_data = PoseDimensionCalculator(processed_data,include_z=True)
new_data.data.to_csv(f"{processed_dir}/pose_data_res2.csv", index=False)

processed_dir = "data/test/processed/fireman/03/"
processed_data = pd.read_csv(f"{processed_dir}/pose_data_raw.csv")
y_cols = [col for col in processed_data.columns if col.endswith('_y')]
processed_data[y_cols] = 1 - processed_data[y_cols]
new_data = PoseDimensionCalculator(processed_data,include_z=True)
new_data.data.to_csv(f"{processed_dir}/pose_data_res2.csv", index=False)